In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import geopandas as gpd
import os 
from shapely import wkt # for WKT 轉幾何物件
from shapely.geometry import LineString, Point
from shapely.ops import substring

# 00 Setup
def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return os.path.abspath(folder_name)

def findfiles(filefolderpath, filetype='.csv', recursive=True):
    """
    尋找指定路徑下指定類型的檔案，並返回檔案路徑列表。

    Args:
        filefolderpath (str): 指定的檔案路徑。
        filetype (str, optional): 要尋找的檔案類型，預設為 '.csv'。
        recursive (bool, optional): 是否檢索所有子資料夾，預設為 True；反之為False，僅查找當前資料夾的所有file。

    Returns:
        list: 包含所有符合條件的檔案路徑的列表。
    """
    filelist = []

    if recursive:
        # 遍歷資料夾及其子資料夾
        for root, _, files in os.walk(filefolderpath):
            for file in files:
                if file.endswith(filetype):
                    file_path = os.path.join(root, file)
                    filelist.append(file_path)
    else:
        # 僅檢索當前資料夾
        for file in os.listdir(filefolderpath):
            file_path = os.path.join(filefolderpath, file)
            if os.path.isfile(file_path) and file.endswith(filetype):
                filelist.append(file_path)

    return filelist

def read_combined_dataframe(file_list, filepath = True):
    dataframes = []
    
    for file in file_list:
        try:
            if file.endswith('.csv'):
                df = pd.read_csv(file)
            elif file.endswith('.shp'):
                df = gpd.read_file(file)
            elif file.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(file)
            else:
                print(f"Unsupported file format: {file}")
                continue
            if filepath:
                df['FilePath'] = file  # 添加來源檔案路徑欄位
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # 合併所有 DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

def dataframe_to_point(df, lon_col, lat_col, crs="EPSG:4326", target_crs="EPSG:3826"):
    '''
    Parameters:
    df (dataframe) : 含經緯度座標欄位的dataframe
    lon_col (str) : 緯度欄位
    Lat_col (str) : 經度欄位
    crs (str) : 目前經緯度座標的座標系統，常用的為4326(WGS84)、3826(TWD97)
    target_crs：目標轉換的座標系統
    '''

    # from shapely.geometry import Point
    # import pandas as pd
    # import geopandas as gpd
    # Create Point geometries from the longitude and latitude columns
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    # Create a GeoDataFrame with the original CRS
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    # Convert the GeoDataFrame to the target CRS
    gdf = gdf.to_crs(epsg=target_crs.split(":")[1])
    return gdf

# 01 讀取TDX資料
def read_bus_stop_of_route_xml(xml_path: str) -> pd.DataFrame:
    """
    讀取 TDX 公車站序 XML（BusStopOfRoute），回傳整理好的 pandas DataFrame。
    
    每一列 = 一個站牌（Stop），同時附上路線 / 營運業者資訊。
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 自動從 root 解析出 namespace（避免寫死）
    if root.tag.startswith("{"):
        uri = root.tag.split("}")[0].strip("{")
    else:
        uri = "https://ptx.transportdata.tw/standard/schema/"
    ns = {"ns": uri}

    def gettext(elem, path):
        """安全取 text，找不到就回 None"""
        if elem is None:
            return None
        child = elem.find(path, ns)
        return child.text if child is not None else None

    rows = []

    # 每一個 <BusStopOfRoute> 代表一條路線 + 方向
    for bsr in root.findall("ns:BusStopOfRoute", ns):

        # 路線共同欄位
        base = {
            "RouteUID":          gettext(bsr, "ns:RouteUID"),
            "RouteID":           gettext(bsr, "ns:RouteID"),
            "RouteName_Zh":      gettext(bsr, "ns:RouteName/ns:Zh_tw"),
            "RouteName_En":      gettext(bsr, "ns:RouteName/ns:En"),
            "SubRouteUID":       gettext(bsr, "ns:SubRouteUID"),
            "SubRouteID":        gettext(bsr, "ns:SubRouteID"),
            "SubRouteName_Zh":   gettext(bsr, "ns:SubRouteName/ns:Zh_tw"),
            "SubRouteName_En":   gettext(bsr, "ns:SubRouteName/ns:En"),
            "Direction":         gettext(bsr, "ns:Direction"),
            "City":              gettext(bsr, "ns:City"),
            "CityCode":          gettext(bsr, "ns:CityCode"),
            "OperatorID":        gettext(bsr, "ns:Operators/ns:Operator/ns:OperatorID"),
            "OperatorName_Zh":   gettext(bsr, "ns:Operators/ns:Operator/ns:OperatorName/ns:Zh_tw"),
            "OperatorNo":        gettext(bsr, "ns:Operators/ns:Operator/ns:OperatorNo"),
        }

        # 底下所有 <Stop>
        for stop in bsr.findall("ns:Stops/ns:Stop", ns):
            row = base.copy()
            row.update({
                "StopUID":          gettext(stop, "ns:StopUID"),
                "StopID":           gettext(stop, "ns:StopID"),
                "StopName_Zh":      gettext(stop, "ns:StopName/ns:Zh_tw"),
                "StopName_En":      gettext(stop, "ns:StopName/ns:En"),
                "StopBoarding":     gettext(stop, "ns:StopBoarding"),
                "StopSequence":     gettext(stop, "ns:StopSequence"),
                "PositionLon":      gettext(stop, "ns:StopPosition/ns:PositionLon"),
                "PositionLat":      gettext(stop, "ns:StopPosition/ns:PositionLat"),
                "GeoHash":          gettext(stop, "ns:StopPosition/ns:GeoHash"),
                "StationID":        gettext(stop, "ns:StationID"),
                "StationGroupID":   gettext(stop, "ns:StationGroupID"),
                "LocationCityCode": gettext(stop, "ns:LocationCityCode"),
            })
            rows.append(row)

    df = pd.DataFrame(rows)

    # 可選：把數值欄位轉型（如果你需要的話）
    for col in ["StopSequence"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="ignore")
    for col in ["PositionLon", "PositionLat"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="ignore")

    return df

def read_bus_shape_of_route_xml(xml_path: str) -> pd.DataFrame:
    """
    讀取 TDX 公車路線 XML（BusShape），回傳整理好的 pandas DataFrame。
    """

    # 解析 XML
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 宣告 XML namespace（必須！）
    ns = {'ns': "https://ptx.transportdata.tw/standard/schema/"}

    records = []

    # 每一個 <BusShape> 就是一筆資料
    for bus in root.findall('ns:BusShape', ns):
        record = {
            "Geometry": bus.findtext('ns:Geometry', namespaces=ns),
            "EncodedPolyline": bus.findtext('ns:EncodedPolyline', namespaces=ns),
            "RouteUID": bus.findtext('ns:RouteUID', namespaces=ns),
            "RouteID": bus.findtext('ns:RouteID', namespaces=ns),
            "RouteName_Zh": bus.find('ns:RouteName/ns:Zh_tw', ns).text if bus.find('ns:RouteName/ns:Zh_tw', ns) is not None else None,
            "RouteName_En": bus.find('ns:RouteName/ns:En', ns).text if bus.find('ns:RouteName/ns:En', ns) is not None else None,
            "SubRouteUID": bus.findtext('ns:SubRouteUID', namespaces=ns),
            "SubRouteID": bus.findtext('ns:SubRouteID', namespaces=ns),
            "SubRouteName_Zh": bus.find('ns:SubRouteName/ns:Zh_tw', ns).text if bus.find('ns:SubRouteName/ns:Zh_tw', ns) is not None else None,
            "SubRouteName_En": bus.find('ns:SubRouteName/ns:En', ns).text if bus.find('ns:SubRouteName/ns:En', ns) is not None else None,
            "Direction": bus.findtext('ns:Direction', namespaces=ns),
            "UpdateTime": bus.findtext('ns:UpdateTime', namespaces=ns),
            "VersionID": bus.findtext('ns:VersionID', namespaces=ns),
        }
        records.append(record)

    # 轉成 DataFrame
    df = pd.DataFrame(records)

    return df


# 01 檢查路線

# 02 拆分路線
def snap_points_to_line(stops_gdf, routes_gdf, 
                        route_id_col, route_direction_col, 
                        seq_id_col, seq_direction_col, 
                        seq_lat_col, seq_lng_col):
    """
    將公車站點 (stops_gdf) 投影到公車路線 (routes_gdf) 上，並動態帶入欄位名稱。
    Parameters:
        stops_gdf (GeoDataFrame): 包含公車站點的 GeoDataFrame。
        routes_gdf (GeoDataFrame): 包含公車路線的 GeoDataFrame。
        route_id_col (str): 路線名稱欄位名稱。
        route_direction_col (str): 路線方向欄位名稱。
        seq_id_col (str): 站點路線名稱欄位名稱。
        seq_direction_col (str): 站點方向欄位名稱。
        seq_lat_col (str): 站點緯度欄位名稱。
        seq_lng_col (str): 站點經度欄位名稱。
    Returns:
        GeoDataFrame: 更新後的公車站點 GeoDataFrame，其中 geometry 已投影到路線。
    """
    snapped_points = []

    for _, stop in stops_gdf.iterrows():
        # 找到與站點路線名稱和方向相符的路線
        matching_route = routes_gdf[(routes_gdf[route_id_col] == stop[seq_id_col]) & 
                                    (routes_gdf[route_direction_col] == stop[seq_direction_col])]

        if not matching_route.empty:
            # 取出該路線的 geometry
            line = matching_route.iloc[0].geometry
            # 計算站點投影到該路線的最近點
            snapped_point = line.interpolate(line.project(stop.geometry))
            snapped_points.append(snapped_point)
        else:
            # 如果沒有匹配的路線，保持原點
            snapped_points.append(stop.geometry)

    # 更新站點的 geometry
    stops_gdf = stops_gdf.copy()
    stops_gdf['geometry'] = snapped_points
    stops_gdf[seq_lat_col] = stops_gdf.geometry.y
    stops_gdf[seq_lng_col] = stops_gdf.geometry.x
    return stops_gdf

def split_routes(busroute_select, 
                 seq_select,
                 route_id_col='RouteName',
                 route_direction_col='Direction',
                 seq_id_col='RouteName',
                 seq_direction_col='Direction',
                 seq_seq_col='Seq',
                 seq_lat_col='Lat',
                 seq_lng_col='Lon'):
    """
    將公車路線 (busroute_select) 依據提供的站序 (seq_select) 上，分為數段的shp。
    Parameters:
        busroute_select (GeoDataFrame): 包含公車路線名稱的 GeoDataFrame。
        seq_select (DataFrame): 包含公車路線站序的 DataFrame。
        seq_routename_col (str): 路線名稱欄位名稱。
        seq_direction_col (str): 路線方向欄位名稱。
        seq_seq_col (str): 站點方向欄位名稱。
        seq_lat_col (str): 站點緯度欄位名稱。
        seq_lng_col (str): 站點經度欄位名稱。
    Returns:
        GeoDataFrame: 更新後的公車站點 GeoDataFrame，其中 geometry 已投影到路線。
    """

    output = []

    for _, route in busroute_select.iterrows():
        route_id = route[route_id_col]
        direction = route[route_direction_col]
        geometry = route['geometry']

        # 過濾對應路線與方向的站點
        stops = seq_select[(seq_select[seq_id_col] == route_id) & 
                           (seq_select[seq_direction_col] == direction)].sort_values(seq_seq_col)

        # 確保站點順序對應於路線
        stop_coords = [(row[seq_lng_col], row[seq_lat_col]) for _, row in stops.iterrows()]

        for i in range(len(stop_coords) - 1):
            start_point = Point(stop_coords[i])
            end_point = Point(stop_coords[i + 1])

            # 找到站點在路線中的比例位置
            start_distance = geometry.project(start_point)
            end_distance = geometry.project(end_point)

            # 提取路線幾何分段
            segment = substring(geometry, start_distance, end_distance)

            output.append({
                'ID': route_id,
                'Direction': direction,
                'StartSeq': stops.iloc[i][seq_seq_col],
                'EndSeq': stops.iloc[i + 1][seq_seq_col],
                'geometry': segment
            })

    return gpd.GeoDataFrame(output)

In [ ]:
# 01-01 讀取站序xml

# 讀取所有站序 xml，轉存為 csv
busstopseq_folder = os.path.join(os.getcwd(), '..', "00_TDX資料下載", "01公車站序資料")
xml_files = findfiles(busstopseq_folder, filetype='.xml', recursive=False)
for xmlfile in xml_files:
    df = read_bus_stop_of_route_xml(xmlfile)
    df.to_csv(xmlfile.replace('.xml', '.csv'), index=False, encoding='utf-8-sig')

# 整併所有的csv
df_seq = read_combined_dataframe(findfiles(busstopseq_folder, filetype='.csv', recursive=False))


C:\Users\kjchang\AppData\Local\Temp\ipykernel_21992\3494255386.py:162: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\kjchang\AppData\Local\Temp\ipykernel_21992\3494255386.py:165: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


In [7]:
# 01-02 讀取路線xml資料
busroute_folder = os.path.join(os.getcwd(), '..', "00_TDX資料下載", "02公車路線資料")
xml_files = findfiles(busroute_folder, filetype='.xml', recursive=False)
for xmlfile in xml_files:
    df = read_bus_shape_of_route_xml(xmlfile)
    df.to_csv(xmlfile.replace('.xml', '.csv'), index=False, encoding='utf-8-sig')

# 整併所有的csv
df_route = read_combined_dataframe(findfiles(busroute_folder, filetype='.csv', recursive=False))

In [ ]:
df_seq = pd.read_csv(r"D:\B-Project\2025\6800\Technical\12票證資料\TicketAnalysis\00_TDX資料下載\01公車站序資料\公車站序資料_新北市_2025-11-21.csv")
df_route = pd.read_csv(r"D:\B-Project\2025\6800\Technical\12票證資料\TicketAnalysis\00_TDX資料下載\02公車路線資料\市區公車路線資料_新北市_2025-11-21.csv")

In [ ]:
# df_route["SubRouteUID"] = df_route["SubRouteUID"].fillna(df_route["RouteUID"])

df_route["SubRouteUID"] = df_route["SubRouteUID"].fillna(
    df_route["RouteUID"].astype(str) + "0"
)


In [ ]:
# 取出兩邊的 SubRouteUID（轉成 set 好比較）
route_set = set(df_route['SubRouteUID'])
seq_set = set(df_seq['SubRouteUID'])

only_in_route = route_set - seq_set
only_in_seq = seq_set - route_set
in_both = route_set & seq_set

print("只在 df_route 出現的 SubRouteUID：")
print(len(only_in_route))

print("\n只在 df_seq 出現的 SubRouteUID：")
print(len(only_in_seq))

print("\n兩邊都有的 SubRouteUID：")
print(len(in_both))


In [ ]:
df_seq[df_seq['SubRouteUID'].isin(only_in_seq)] [['RouteUID', 'RouteName_Zh', 'SubRouteUID', 'SubRouteName_Zh']].drop_duplicates()  

In [ ]:
# 03 黏貼站序道路線上
# 將 Geometry 欄位的 WKT 轉為 shapely geometry
df_route["Geometry"] = df_route["Geometry"].apply(wkt.loads)

# 建立 GeoDataFrame
gdf_route = gpd.GeoDataFrame(df_route, geometry="Geometry", crs="EPSG:4326")

In [ ]:
# gdf_route = gdf_route[gdf_route['SubRouteUID'].isin(in_both)].reset_index(drop=True)
# df_seq = df_seq[df_seq['SubRouteUID'].isin(in_both)].reset_index(drop=True)
gdf_seq = dataframe_to_point(df_seq, lon_col='PositionLon', lat_col='PositionLat', crs="EPSG:4326", target_crs="EPSG:4326")
gdf_route.rename(columns={'Geometry':'geometry'}, inplace=True)

In [ ]:
# 02 拆分路線
routesegment_folder = create_folder(os.path.join(os.getcwd(), '..', "03_處理後資料", "01_公車路線依站序拆分"))

# 02-1 將公車站序點位投影到路線上
gdf_snapstop = snap_points_to_line(gdf_seq, gdf_route,
                                    route_id_col='SubRouteUID',
                                    route_direction_col='Direction',
                                    seq_id_col='SubRouteUID',
                                    seq_direction_col='Direction',
                                    seq_lat_col='PositionLat',
                                    seq_lng_col='PositionLon')

# 02-2 依站序拆分路線
gdf_routesegment = split_routes(gdf_route, gdf_snapstop,
                                route_id_col='SubRouteUID',
                                route_direction_col='Direction',
                                seq_id_col='SubRouteUID',
                                seq_direction_col='Direction',
                                seq_seq_col='StopSequence',
                                seq_lat_col='PositionLat',
                                seq_lng_col='PositionLon')

gdf_routesegment.rename(columns = {'ID':'SubRouteUID'}, inplace=True)

# 輸出
routesegment_filepath = os.path.join(routesegment_folder, '新北市公車拆分.shp')

gdf_routesegment.to_file(routesegment_filepath, 
                         index=False)

In [ ]:
gdf_routesegment['SubRouteUID'].unique().

In [ ]:
# 把站點投影到路線上

# 用站點將路線進行拆分


gdf_routesegment.to_file(r'D:\B-Project\2025\6800\Technical\12票證資料\TicketAnalysis\00_TDX資料下載\Trial\新北市拆分路網圖.shp', index=False)

In [ ]:
gdf_seq[['RouteUID','SubRouteUID', 'RouteName_Zh', 'SubRouteName_Zh']].drop_duplicates().sort_values(['SubRouteUID'])

In [ ]:
df_GTSFseq = pd.read_csv(r"D:\B-Project\2025\6800\Technical\12票證資料\其他分析資料\TDX_GTFS_20250617\shapes.txt")
df_GTSFseq['RouteUID'] = df_GTSFseq['shape_id'].str.split('_').str[0]
df_GTSFseq['Direction'] = df_GTSFseq['shape_id'].str.split('_').str[1]
df_GTSFseq.rename(columns = {'shape_id':'ShapeUID', 
                            'shape_pt_lat':'PositionLat',
                            'shape_pt_lon':'PositionLon',
                            'shape_pt_sequence':'StopSequence'}, 
                            inplace=True)

gdf_GTSFseq = dataframe_to_point(df_GTSFseq, lon_col='PositionLon', lat_col='PositionLat', crs="EPSG:4326", target_crs="EPSG:4326")

In [ ]:
gdf_GTSFseq['lens'] = gdf_GTSFseq['RouteUID'].str.len()

In [ ]:
gdf_GTSFseq